In [1]:
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from time import time

import pyspark as ps
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as F

In [16]:

class MovieRecommender():
    """Template class for a Movie Recommender system."""

    def __init__(self, local=False):
        """Constructs a MovieRecommender"""
        self.spark = ps.sql.SparkSession.builder \
              .master("local[4]") \
              .appName("Movie Reccomender") \
              .getOrCreate()
        self.sc = self.spark.sparkContext
        self.logger = logging.getLogger('reco-cs')
        self.users = self.sc.textFile('data/users.dat').map(lambda x: (int(x.split('::')[0]), x))
        self.movies = self.sc.textFile('data/movies.dat').map(lambda x: (int(x.split('::')[0]), x))
        self.local = local



    def fit(self, ratings):
        """
        Trains the recommender on a given set of ratings.

        Parameters
        ----------
        ratings : pandas dataframe, shape = (n_ratings, 4)
                  with columns 'user', 'movie', 'rating', 'timestamp'

        Returns
        -------
        self : object
            Returns self.
        """
        self.logger.debug("starting fit")
        
        #Save the training data for later use:
        
        self.training_data = ratings.toPandas()
        self.users_train_unique = self.training_data.user.unique()
        self.movies_train_unique = self.training_data.movie.unique()
        
        #Begin Transforming the data for fitting
        t0 = time()
        users = self.users
        movies = self.movies
        
        # Maps the ratings df structure to that of the test data'a
        ratings = ratings.rdd.map(tuple)
        ratings = ratings.map(lambda x: '::'.join(x))
        ratings = ratings.map(lambda x: (int(x.split('::')[0]), x))
        self.ratings = ratings
        
        # Joins all the tables together for training
        joined = ratings.join(users)
        temp = joined.map(lambda x: '::'.join(x[1])).map(lambda x: (int(x.split('::')[1]), x))
        joined_full = temp.join(movies).map(lambda x: '::'.join(x[1]))

        # Removes the :: seperator from the RDD
        def split_to_cols(x):
            values = x.split('::')
            return (int(values[0]), int(values[1]), int(values[2]))

        # Not used but kept around because it could be 
        def get_ratings(x):
            values = x.split('::')
            return (int(values[2]))

        # Turns the RDD into a DataFrame
        spark_df = joined_full.map(split_to_cols)

        schema = StructType([
            StructField("userID", IntegerType(), True),
            StructField("movieID", IntegerType(), True),
            StructField("rating", IntegerType(), True)])
        
        # Creates the proper train DataFrame for fitting
        train = self.spark.createDataFrame(spark_df, schema)

        # Instantiate the model (Alternating Least Squares)
        als = ALS(
                itemCol='movieID',
                userCol='userID',
                ratingCol='rating',
                nonnegative=True,    
                regParam=0.4,
                maxIter=10,
                rank=14)
        
        # Creates the reccomender by fitting the training data
        self.recommender = als.fit(train)
        
        # Fit the model
        print('Model Created. Trainging....')
        self.recommender = als.fit(train)
        self.fitted = True
        
        self.logger.debug("finishing fit")
        print('DONE! ', time()-t0, ' seconds.')
        return(self)


    def transform(self, requests):
        """
        Predicts the ratings for a given set of requests.

        Parameters
        ----------
        requests : pandas dataframe, shape = (n_ratings, 2)
                  with columns 'user', 'movie'

        Returns
        -------
        dataframe : a pandas dataframe with columns 'user', 'movie', 'rating'
                    column 'rating' containing the predicted rating
        """

        test_df = requests.toPandas()
        self.test_df = test_df
        
        #Filter down the request data
        self.old_old = test_df[(test_df.user.isin(self.users_train_unique))
                          & (test_df.movie.isin(self.movies_train_unique))]
        newish = test_df[~((test_df.user.isin(self.users_train_unique))
                         & (test_df.movie.isin(self.movies_train_unique)))]
        self.newish = newish
        
        #Split off the new users/movies:
        self.requests_new_movies = newish[(newish.user.isin(self.users_train_unique))
                                    & ~(newish.movie.isin(self.movies_train_unique))]
        
        self.requests_new_users = newish[~((newish.user.isin(self.users_train_unique))
                                    & ~(newish.movie.isin(self.movies_train_unique)))]
        
        requests = self.spark.createDataFrame(self.old_old)
        
        self.logger.debug("starting predict")
        self.logger.debug("request count: {}".format(requests.count()))
        t0 = time()
        users = self.users
        movies = self.movies
        
        # Gets the requests in the right shape
        requests = requests.rdd.map(tuple)
        requests = requests.map(lambda x: '::'.join(x))
        requests = requests.map(lambda x: (int(x.split('::')[0]), x))
        
        joined = requests.join(users)
        temp = joined.map(lambda x: '::'.join(x[1])).map(lambda x: (int(x.split('::')[1]), x))

        joined_full = temp.join(movies).map(lambda x: '::'.join(x[1]))

        def split_to_cols(x):
            values = x.split('::')
            return (int(values[0]), int(values[1]), int(values[2]))

        def get_ratings(x):
            values = x.split('::')
            return (int(values[2]))

        data_rdd = joined_full.map(split_to_cols)
        j_ratings = joined_full.map(get_ratings)

        schema = StructType([
            StructField("userID", IntegerType(), True),
            StructField("movieID", IntegerType(), True),
            StructField("rating", IntegerType(), True)])
        
        test = self.spark.createDataFrame(data_rdd, schema)
        
        self.logger.debug("finishing predict for recognized users and movies")
        
        print('Transforming...')
        output = self.recommender.transform(test)
        
        output = output.toPandas()
        output.drop('rating',axis=1,inplace=True)
        output.rename(columns={'userID':'user', 'movieID':'movie'}, inplace = True)
        print('DONE! ', time()-t0, ' seconds.')
        
        print("Sending the new users to different model..")
        t0 = time()
        self.new_user_pred = self.weighted_Recommendation()
        output = pd.concat([output,self.new_user_pred],axis=0)
        print('DONE! ', time()-t0, ' seconds.')
        
        
        print("Sending the new movies to different model..")
        t0 = time()
        if self.local == False:
            self.new_movie_pred = self.requests_new_movies.copy()
            self.new_movie_pred['prediction'] = 2.5
            output = pd.concat([output,self.new_movie_pred],axis=0)
        else: 
            #LOAD IN SIMILARITY MATRIX
            Sims = pd.read_csv('data/matrix_overview.tar.xz', compression='infer')
            Sims = Sims.rename(mapper={'matrix_overview.csv':'ID'}, axis=1,copy=False).dropna()
            Sims.ID = Sims.ID.astype(int)
            Sims = Sims.set_index('ID')
            
            preds = []
            for i,row in self.requests_new_movies.copy().iterrows():
                user = row.user
                movie = row.movie
                preds.append(self.pred_on_similarity(self.training_data, Sims, user, movie))
            self.new_movie_pred['prediction'] = pd.Series(preds)
            
        
        
        print('DONE! ', time()-t0, ' seconds.')
        return(output)
    
    
    def weighted_Recommendation(self, is_sparse=False):
        pd.options.display.float_format = '{:,.2f}'.format
        training = self.training_data.copy()
        users_movies = self.requests_new_users
        
        if is_sparse:

            grouped_training = pd.DataFrame(np.full(len(training.columns),2.5))
            grouped_training['movie'] = np.array(training.columns)
            grouped_training['rating']= np.array(training.mean(axis = 0))
            grouped_training['vote']= np.array(training.count(axis = 0))
            grouped_training = grouped_training[['movie','rating','vote']]

        else:
            training['rating'] = training['rating'].astype(int)
            grouped_training = training.groupby('movie') \
                        .agg({'user':'size', 'rating':'mean'}) \
                        .rename(columns={'user':'vote','rating':'rating'}) \
                        .reset_index()


        # Calculate the minimum number of voters required to be in the chart
        m = grouped_training['vote'].quantile(0.5)

        # Filter out all qualified movies into a new DataFrame
        scorings = grouped_training.copy().loc[grouped_training['vote'] >= m]

        F = pd.merge(users_movies, scorings, on='movie', how='left')
        F['rating'].fillna(2.5, inplace=True)
        final = F[['user','movie','rating']]
        final.rename(columns={'rating':'prediction'},inplace=True,copy=False)

        return(final)
                            
    def pred_on_similarity(df, similarity_matrix, userID, movieID, num_similar=3):
        '''
        GENERATE 1 PREDICTED VALUE OF AN UNSEEN MOVIE FOR AN EXISTING USER BASED ON THAT USER'S RATINGS OF THE MOST 
        SIMILAR MOVIES TO THE MOVIE IN QUESTION.


        df : 'pandas dataframe with columns user(int), movie(int)
        similarity_matrix : square matrix pd.DataFrame of similarities 
        userID : int : id of user in df
        movieID : int/str : id of movie in df
        num_similary : int : compare movie in question to *num_similar* number of other movies the user has rated.
        '''
        n = num_similar
        movieID = str(movieID)
        userID = str(userID)
        print(df.head())
        user = df[df.user == userID][['movie','rating']] #get user movies and ratings by the user in question
        m = similarity_matrix[movieID].reset_index() #get similarities for the movie in question
        m.columns = ['movie','similarity'] #rename columns for merge

        #fix formatting
        m.movie = m.movie.astype(int)
        user.movie = user.movie.astype(int)

        merged = m.merge(user, on='movie',how='inner') #merge movie similarities with ratings

        #fix formatting 
        merged.rating = merged.rating.astype(float)
        merged.similarity = merged.similarity.astype(float)

        merged['product'] = merged.rating*merged.similarity #calculate rating*similarity

        #get top similarity value for normalizing
        sorted_sims = merged.similarity.sort_values(ascending=False)
        norm = sorted_sims[sorted_sims < 1].iloc[0]

        #sort by top similarities, take first n ratings*similarities, take average, normalize
        p = np.mean(merged.sort_values(by='similarity', ascending=False)['product'][:n])/norm
        return p 

In [3]:
# train_data = pd.read_csv('data/training.csv')

In [3]:
spark = ps.sql.SparkSession.builder\
                                .master("local[4]")\
                                .appName("Reccomenders Paired")\
                                .getOrCreate()
sc = spark.sparkContext

In [4]:
request_data = spark.read.csv('data/requests.csv', header=True)
train_data = spark.read.csv('data/training.csv', header=True)

In [20]:
rec = MovieRecommender(local=False)
model = rec.fit(train_data)
model.fitted

Model Created. Trainging....
DONE!  12.87046194076538  seconds.


True

In [21]:
pred = model.transform(request_data)

Transforming...
DONE!  8.873837947845459  seconds.
Sending the new users to different model..
DONE!  0.4452848434448242  seconds.
Sending the new movies to different model..
DONE!  0.014931917190551758  seconds.


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [22]:
pred.to_csv('output_1.csv',index=False)

In [132]:
model.new_user_pred.head()

,user,movie,prediction
0,403,1036,4.15
1,424,2581,3.15
2,229,2839,2.50
3,229,3089,4.40
4,424,2324,4.33


In [139]:
pred.shape

(200209, 3)

In [140]:
print(model.test_df.shape)
print(model.old_old.shape)
print(model.newish.shape)
print(model.requests_new_movies.shape)
print(model.requests_new_users.shape)

(200209, 2)
(104439, 2)
(95770, 2)
(91, 2)
(95679, 2)


In [36]:
pred_df = pred.toPandas()

In [72]:
pred_df.head()

,userID,movieID,rating,prediction
0,4169,148,4169,2.720469
1,5333,148,5333,2.666358
2,4387,148,4387,2.318244
3,840,148,840,3.150011
4,752,148,752,2.133929


In [7]:
# train_df = train_data.toPandas()
# test_df = request_data.toPandas()

train_df = pd.read_csv('data/requests.csv')
test_df = pd.read_csv('data/training.csv')

In [43]:
pred_df[pred_df.prediction.isna()].shape

(0, 4)

In [44]:
pred_df.shape[0]

104439

In [45]:
users_train = train_df.user.unique()
movies_train = train_df.movie.unique()

In [46]:
check = pred_df[pred_df.userID.isin(users_train)]
check = check[check.movieID.isin(movies_train)]
check.isnull().sum()

userID        0
movieID       0
rating        0
prediction    0
dtype: int64

In [50]:
# IF CASES WHERE BOTH 
check = pred_df[(pred_df.userID.isin(users_train)) & (~pred_df.movieID.isin(movies_train))]
check.isnull().sum()

userID        0
movieID       0
rating        0
prediction    0
dtype: int64

userID        0
movieID       0
rating        0
prediction    0
dtype: int64

# Max's stuff

In [ ]:
pred.show()

In [ ]:
pred_df = pred.toPandas()

In [ ]:
train_df = train_data.toPandas()
test_df = request_data.toPandas()

In [ ]:
train_df.head()

In [ ]:
test_df.shape[0]

In [ ]:
nas = pred_df[pred_df.prediction.isna()]

In [ ]:
unique_users = train_df['user'].unique()
unique_users_na_test = nas['user'].unique()

unique_movie = train_df['movie'].unique()
unique_movie_na_test = nas['movie'].unique()

In [ ]:
len(set(unique_movie).intersection(set(unique_movie_na_test)))

In [ ]:
len(set(unique_users).intersection(set(unique_users_na_test)))

In [129]:
unique_users_na_test.shape

(708,)

In [136]:
type(unique_users)

numpy.ndarray

In [138]:
nans

ValueError: Lengths must match to compare

In [134]:
nas[(nas['user'] in unique_users) & (nas['movie'] in unique_movie)]

ValueError: Lengths must match to compare

In [98]:
nas.rename({'userID':'user', 'movieID':'movie'},copy=False, inplace = True,axis=1)

In [121]:
nas['user'] = nas['user'].astype(int)
nas['movie'] = nas['movie'].astype(int)
train_df['user'] = train_df['user'].astype(int)
train_df['movie'] = train_df['movie'].astype(int)
nas.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,user,movie,rating,prediction
0,53,148,53,NaN
5,216,148,216,NaN
6,482,148,482,NaN
8,424,148,424,NaN
11,26,463,26,NaN


In [122]:
pd.merge(train_df, nas,on='user',how='inner').shape

(43389, 7)

In [11]:
# spark = ps.sql.SparkSession.builder\
#                                 .master("local[4]")\
#                                 .appName("Reccomenders Paired")\
#                                 .getOrCreate()
# sc = spark.sparkContext

# users = sc.textFile('data/users.dat').map(lambda x: (int(x.split('::')[0]), x))
# movies = sc.textFile('data/movies.dat').map(lambda x: (int(x.split('::')[0]), x))


# ratings = spark.read.csv('data/training.csv', header=True)


# ratings = ratings.rdd.map(tuple)
# ratings = ratings.map(lambda x: '::'.join(x))
# ratings = ratings.map(lambda x: (int(x.split('::')[0]), x))

# users.take(5), ratings.take(5)

# joined = ratings.join(users)
# # index is the user id of that rating 
# joined.take(5)

# temp = joined.map(lambda x: '::'.join(x[1])).map(lambda x: (int(x.split('::')[1]), x))

# # index is now the movie id of that rating
# temp.take(5)

# joined_full = temp.join(movies).map(lambda x: '::'.join(x[1]))
# joined_full.take(5)
# joined_full


# def split_to_cols(x):
#     values = x.split('::')
#     return (int(values[0]), int(values[1]), int(values[2]))

# def get_ratings(x):
#     values = x.split('::')
#     return (int(values[2]))

# data_rdd = joined_full.map(split_to_cols)
# data_rdd.take(5)

# j_ratings = joined_full.map(get_ratings)
# j_ratings.take(5)

# schema = StructType([
#     StructField("userID", IntegerType(), True),
#     StructField("movieID", IntegerType(), True),
#     StructField("rating", IntegerType(), True)])
# data = spark.createDataFrame(data_rdd, schema)

# train, test = data.randomSplit([0.8, 0.2], seed=420)
# train.show()

# als = ALS(
#     itemCol='movieID',
#     userCol='userID',
#     ratingCol='rating',
#     nonnegative=True,    
#     regParam=0.4,
# maxIter=10,
#     rank=14)

# recommender = als.fit(train)


predictions = recommender.transform(test)
predictions.show()



predictions_df = predictions.toPandas()
train_df = train.toPandas()
predictions_df = predictions.toPandas().fillna(train_df['rating'].mean())
predictions_df['r_squared'] = (predictions_df['rating'] - predictions_df['prediction'])**2


RMSE = np.sqrt(sum(predictions_df['r_squared']) / len(predictions_df))
RMSE


+------+-------+------+
|userID|movieID|rating|
+------+-------+------+
|   636|    968|     5|
|   636|   1240|     5|
|   636|   1320|     4|
|   636|   1544|     5|
|   636|   1584|     3|
|   636|   2288|     2|
|   636|   2456|     2|
|   636|   3016|     3|
|   637|    440|     3|
|   637|    480|     3|
|   637|    520|     4|
|   637|    592|     2|
|   637|    920|     5|
|   637|    952|     3|
|   637|    968|     5|
|   637|   1080|     3|
|   637|   1096|     5|
|   637|   1200|     3|
|   637|   1208|     3|
|   637|   1224|     5|
+------+-------+------+
only showing top 20 rows



0.9991700955071164